 # Import the RPB model

In [1]:
from RPB_model import *
from pyomo.environ import *
from pyomo.environ import SolverManagerFactory

# Create a full RPB model (Only need to run once)

In [2]:
RPB = full_model_creation(lean_temp_connection=True, configuration = "counter-current", has_pressure_drop=False)

# Create Objective Function

In [3]:
# add objective
@RPB.Expression()
def obj(RPB):
    return RPB.energy_requirement

RPB.objective = Objective(expr=RPB.obj)

RPB.objective.pprint()

objective : Size=1, Index=None, Active=True
    Key  : Active : Sense    : Expression
    None :   True : minimize :        obj


### Create solver object (Only need to run once)

In [7]:
# results = SolverFactory("gams").solve(
#     RPB,
#     tee=True,
#     keepfiles=True,
#     solver="conopt4",
#     tmpdir="temp",
#     add_options=["gams_model.optfile=1;"],
# )

# solver = SolverFactory("ipopt")
# solver.options = {
#         "max_iter": 1000,
#         "bound_push": 1e-8,
#         # "halt_on_ampl_error": "yes",
#         "tol": 1e-4,
#         "max_cpu_time": 5*60,
#         # "mu_strategy": "adaptive",
#         }

# Set Bounds for Decision Variables
##### Should only need to run when initializing the model

In [4]:
RPB.ads.L.setlb(0.01)
RPB.ads.L.setub(40)
RPB.des.L.setlb(0.01)
RPB.des.L.setub(40)

RPB.ads.Tx.setlb(25+273)
RPB.ads.Tx.setub(95+273)

RPB.des.Tx.setlb(100+273)
RPB.des.Tx.setub(160+273)

RPB.ads.P_in.setub(1.5)
RPB.ads.P.setub(1.5)

RPB.des.P_in.setub(1.5)
RPB.des.P_in.setlb(1.01325)
RPB.des.P.setub(1.5)

RPB.ads.w_rpm.setlb(0.00001)
RPB.ads.w_rpm.setub(0.1)

# Initialize Using Json File (90% Capture)
## Only run first time or if solver fails to converge to an optimal solution
#### May have to scroll to see all results

In [5]:
from_json(RPB, fname="polishing step optimized solution 031824.json.gz", gz=True)

{'etime_load_file': 0.17000484466552734,
 'etime_read_dict': 0.5572657585144043,
 'etime_read_suffixes': 0.05087685585021973}

# Run to get RPB Results
### Can copy and paste the table to excel (formatting might just be weird)

In [6]:
report(RPB)

,Value,Doc,Fixed,Lower Bound,Upper Bound
ads.L,2.296788,Bed Length [m],False,0.01,40
ads.D,10.000000,Bed diameter [m],True,None,None
ads.w_rpm,0.004795,bed rotational speed [revolutions/min],False,0.00001,0.1
ads.theta,0.606012,Fraction of bed [-],False,0.01,0.99
des.theta,0.393988,Fraction of bed [-],False,0.01,0.99
ads.P_in,1.500000,Inlet flue gas pressure [bar],False,1,1.5
ads.P_out,1.013250,Outlet adsorber pressure [bar],True,0.99,1.2
ads.F_in,2572.502253,Inlet adsorber gas flow [mol/s],False,0,None
ads.Tg_in,363.000000,Inlet flue gas temperature [K],True,None,None
ads.Tx,298.000000,"heat exchange fluid temperature, constant [K]",False,298,368


# Fix decision Variables

In [7]:
RPB.ads.F_in.fix()
RPB.ads.P_in.fix()
RPB.ads.P_out.unfix()

# Ipopt Solver

In [ ]:
solver = SolverFactory("ipopt")
solver.options = {
        "max_iter": 1000,
        "bound_push": 1e-8,
        # "halt_on_ampl_error": "yes",
        "tol": 1e-4,
        # "max_cpu_time": 5*60,
        # "mu_strategy": "adaptive",
        }
results = solver.solve(RPB, tee=True)
results.write()

component keys that are not exported as part of the NL file.  Skipping.
that are not Var, Constraint, Objective, or the model.  Skipping.
Ipopt 3.13.2: max_iter=1000
bound_push=1e-08
tol=0.0001


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use 

# Fix Capture Efficiency and Solve

In [15]:
RPB.ads.CO2_capture.fix(0.9)
# solver = SolverFactory('conopt')
# solver.solve(RPB, tee=True)

# solver.solve(RPB, tee=True)
# results = SolverFactory("gams").solve(
#     RPB,
#     tee=True,
#     keepfiles=True,
#     solver="conopt4",
#     tmpdir="temp",
#     add_options=["gams_model.optfile=1;"],
# )

opt = SolverFactory("gams")
solver_manager = SolverManagerFactory('neos')
results = solver_manager.solve(
    RPB,
    tee=True,
    # keedfiles=True,
    solver="conopt",
    # tmpdir="temp",
    options={'outlev': 2,
            'workfactor': 3,},
    # add_options=["gams_model.optfile=1;"],
)
results.write()

component keys that are not exported as part of the NL file.  Skipping.
component keys that are not exported as part of the NL file.  Skipping.
model.name="unknown";
    - termination condition: optimal
    - message from solver: CONOPT 3.17A\x3a Locally optimal; objective
      4.068171879792977; 176 iterations; evals\x3a nf = 1314, ng = 759, nc =
      12345, nJ = 759, nH = 0, nHv = 35; 6 failed evaluations
# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 21934
  Number of variables: 21940
  Sense: unknown
# ----------------------------------------------------------
#   Solver Information
# --------------------

In [3]:
import os
email = os.environ.get('NEOS_EMAIL', '')
print(email)
print(type(email))

ss0233@mix.wvu.edu
<class 'str'>


In [12]:
gams_model.optfile=1;

ModuleNotFoundError: No module named 'temp.gams_model'

# Check Infeasible Constraints

In [18]:
from pyomo.util.infeasible import find_infeasible_constraints as find_inf_con
i = 0
for c in find_inf_con(RPB):
    i += 1
    print(c[0], c[1])
print(f'There are {i} infeasible constraints')

There are 0 infeasible constraints
